In [17]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *

import numpy as np

In [18]:
mesh = Mesh(unit_square.GenerateMesh(maxh=1))

order = 1
fes = HCurl(mesh, order=order, type1=False)
gfu = GridFunction(fes)
gfut = GridFunction(fes, multidim=0)

Here a little testing to see what happens with type1=True and type1=False.
My VSCode Drawing of gridfunctions is very buggy, but it seems like that type1=False leads to the interior of the elements having a function as well.
This because first of all type1=False also leads to the DOF's increasing and one can see it visually.

In [19]:
print(fes.ndof)
settings = {"camera": {"transformations": [
        {"type": "rotateX", "angle": -45}]}}

# for i in range(len(gfu.vec)):

#     gfu.vec[:] = 0
#     gfu.vec[i] = 1

#     gfut.AddMultiDimComponent(gfu.vec)

# Draw(gfut, deformation=False, animate=True, autoscale = True , order = 3,  vectors = { "grid_size":30}, settings=settings);

10


I wanna add testing to see what happens to the crossproduct in 2D, I need this to nicely write down my boundary integrals independent of the dimension

In [20]:
normal = 1*specialcf.normal(mesh.dim)
tangent = 1*specialcf.tangential(mesh.dim)
# z_vector = Cross(normal, tangent) # Doesnt work bcs of the dimensions!, obviously, but thought maybe it works and yields a scalar..

# Draw(z_vector)

Wanna Test some mesh size calculation thingies!

In [21]:
def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2 
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)
    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_h(mesh):
    max_h = 0     
    if mesh.dim == 2:

        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

mesh = Mesh(unit_cube.GenerateMesh(maxh=1))

for i in range(5):
    print(f"Refinement step {i}:")
    
    num_elements = mesh.Elements()
    print(f"Number of elements before refinement: {num_elements}")
    
    h_max_eval= calc_h(mesh)
    print(f"h_max after refinement {i}: {h_max_eval}")
        
    if i < 6:  
        mesh.Refine()
    
    num_elements_after_refine = mesh.Elements()
    print(f"Number of elements after refinement: {num_elements_after_refine}")
    
    if num_elements_after_refine == num_elements:
        print("Warning: No change in the number of elements after refinement!")

Refinement step 0:
Number of elements before refinement: [0,12)
h_max after refinement 0: 0.7500000005423021
Number of elements after refinement: [0,96)
Refinement step 1:
Number of elements before refinement: [0,96)
h_max after refinement 1: 0.37500000098164693
Number of elements after refinement: [0,768)
Refinement step 2:
Number of elements before refinement: [0,768)
h_max after refinement 2: 0.18750000049082352
Number of elements after refinement: [0,6144)
Refinement step 3:
Number of elements before refinement: [0,6144)
h_max after refinement 3: 0.09375000024541177
Number of elements after refinement: [0,49152)
Refinement step 4:
Number of elements before refinement: [0,49152)
h_max after refinement 4: 0.046875000122705936
Number of elements after refinement: [0,393216)


In [22]:
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.25))

# mesh.Refine()
# mesh.Refine()
# mesh.Refine()
# Draw(mesh)

In [23]:
mesh.Refine()
#Draw(mesh)

In [24]:
mesh.Refine()
#Draw(mesh)